In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.preprocessing import image
from keras import applications
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

training_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/Food_crops/train',
        target_size=(224,224),
        batch_size=32,
        shuffle= False,
        class_mode='categorical')

val_set = datagen.flow_from_directory(
        '/content/drive/MyDrive/Food_crops/validate',
        target_size=(224,224),
        batch_size=16,
        shuffle= False,
        class_mode="categorical")

test_set = datagen.flow_from_directory(
        '/content/drive/MyDrive/Food_crops/test',
        target_size=(224,224),
        batch_size=1,
        shuffle= False,
        class_mode="categorical")

In [ ]:
csv_logger = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/Food_crops/training.csv', append=True)

img_inputs = tf.keras.Input(shape=(224, 224, 3))

xa = tf.keras.layers.Conv2D(filters = 64,kernel_size = (3,3), activation="relu")(img_inputs)
xa = tf.keras.layers.Conv2D(filters = 64,kernel_size = (3,3), activation="relu")(xa)
xa = tf.keras.layers.SeparableConv2D(filters = 64,kernel_size = (3,3), activation="relu")(xa)
xa = tf.keras.layers.GlobalMaxPooling2D()(xa)
xa = Dense(64, activation = 'relu')(xa)

xb = tf.keras.layers.Conv2D(filters = 64,kernel_size = (5,5), activation="relu")(img_inputs)
xb = tf.keras.layers.Conv2D(filters = 64,kernel_size = (5,5), activation="relu")(xb)
xb = tf.keras.layers.SeparableConv2D(filters = 64,kernel_size = (5,5), activation="relu")(xb)
xb = tf.keras.layers.GlobalMaxPooling2D()(xb)
xb = Dense(64, activation = 'relu')(xb)

xd = tf.keras.layers.Conv2D(filters = 64,kernel_size = (7,7), activation="relu")(img_inputs)
xd = tf.keras.layers.Conv2D(filters = 64,kernel_size = (7,7), activation="relu")(xd)
xd = tf.keras.layers.SeparableConv2D(filters = 64,kernel_size = (7,7), activation="relu")(xd)
xd = tf.keras.layers.GlobalMaxPooling2D()(xd)
xd = Dense(64, activation = 'relu')(xd)

xc = tf.keras.layers.Average()([xa, xb, xd])

output = Dense(12, activation = 'softmax')(xc)

model = Model(inputs = img_inputs, outputs = output)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# model.summary()

r1 = model.fit(training_set,epochs=50,validation_data = val_set, callbacks = [csv_logger])



In [ ]:
Y_pred = model.predict(test_set)
y_pred = np.argmax(Y_pred, axis=1)


cm = confusion_matrix(test_set.classes, y_pred)
df = pd.DataFrame(cm, index=None)
df.to_excel("/content/drive/MyDrive/Food_crops/Model_confusionmatrix.xlsx")


report = classification_report(test_set.classes, y_pred, output_dict=True)
dff = pd.DataFrame(report).transpose()
dff.to_excel("/content/drive/MyDrive/Food_crops/Model_classificationreport.xlsx")